In [1]:
import pdb
import numpy as np
from astropy.wcs import WCS
from shift import shift_twod
from VieroLibrary.dist_idl import dist_idl
from VieroLibrary import readcol
from lmfit import Parameters, minimize, fit_report
from smoothmap import smoothmap
from astropy.io import fits
from fluctfit import simmap_from_cat_five_pops
#from fluctfit import simmap_from_cat_general
from gauss_kern import gauss_kern
#import pylab as pl
import matplotlib.pyplot as plt
import math
import pylab
from grab_random import grab_random
%matplotlib inline 

In [2]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

In [3]:
path_catalog = '/data/maps_cats_models/catalogs/UVISTA/'
file_catalog = 'UVISTA_final_v4.1_zCOSMOS_zEAZY_RFcolors_IDs_mass_AGN_SFRs.csv'

In [4]:
id0,ra0,dec0,mips24_0,z_peak0,u_v0,v_j0,reddening_AGN0,a_hat_AGN0,F_ratio0,l_uv0,l_ir0,sfr_uv,lmass0 = readcol.readcol(path_catalog+file_catalog,fsep=',',twod=False, skipline=1)
uvj0 = u_v0 ** 2 + v_j0 ** 2

In [5]:
nsrc = len(ra0)
sfg = np.ones(nsrc)
effc = np.ones(nsrc)

Fcut = 25
indsb = np.where((mips24_0 > 300.) & (F_ratio0 < Fcut))
sfg[indsb] = 4

z0 = 9.3 
slope = (12.0-9.3)/(1.25)
indlocal = np.where((sfg != 4) & (lmass0 > z_peak0 * slope + z0) & (mips24_0 > 200.))
sfg[indlocal] = 3

indagn = np.where(F_ratio0 >= Fcut)
sfg[indagn] = 2

for i in range(nsrc):
	if (sfg[i] < 2) and (u_v0[i] > 1.3) and (v_j0[i] < 1.5):
		if (z_peak0[i] < 1):
			if (u_v0[i] > (v_j0[i]*0.88+0.69) ): sfg[i]=0
		if (z_peak0[i] > 1):
			if (u_v0[i] > (v_j0[i]*0.88+0.59) ): sfg[i]=0

print 'done sorting uvj and agn'

done sorting uvj and agn


In [6]:
indsf = np.where(sfg == 1)
indqt = np.where(sfg == 0)
nsfg = len(indsf[0])
nqt = len(indqt[0]) 
nagn = len(indagn[0]) 
nloc = len(indlocal[0])
nsb = len(indsb[0])

id_sf = id0[indsf]
ra_sf = ra0[indsf]
dec_sf = dec0[indsf]
z_peak_sf = z_peak0[indsf]
u_v_sf = u_v0[indsf]
v_j_sf = v_j0[indsf]
uvj_sf = uvj0[indsf]
lmass_sf = lmass0[indsf]
mips24_sf = mips24_0[indsf]

id_qt = id0[indqt]
ra_qt = ra0[indqt]
dec_qt = dec0[indqt]
z_peak_qt = z_peak0[indqt]
u_v_qt = u_v0[indqt]
v_j_qt = v_j0[indqt]
uvj_qt = uvj0[indqt]
lmass_qt = lmass0[indqt]
mips24_qt = mips24_0[indqt]

id_agn = id0[indagn]
ra_agn = ra0[indagn]
dec_agn = dec0[indagn]
z_peak_agn = z_peak0[indagn]
u_v_agn = u_v0[indagn]
v_j_agn = v_j0[indagn]
uvj_agn = uvj0[indagn]
lmass_agn = lmass0[indagn]
f_ratio = F_ratio0[indagn]
a_hat_agn = a_hat_AGN0[indagn]
mips24_agn = mips24_0[indagn]

id_loc = id0[indlocal]
ra_loc = ra0[indlocal]
dec_loc = dec0[indlocal]
z_peak_loc = z_peak0[indlocal]
u_v_loc = u_v0[indlocal]
v_j_loc = v_j0[indlocal]
uvj_loc = uvj0[indlocal]
lmass_loc = lmass0[indlocal]
mips24_loc = mips24_0[indlocal]

id_sb = id0[indsb]
ra_sb = ra0[indsb]
dec_sb = dec0[indsb]
z_peak_sb = z_peak0[indsb]
u_v_sb = u_v0[indsb]
v_j_sb = v_j0[indsb]
uvj_sb = uvj0[indsb]
lmass_sb = lmass0[indsb]
mips24_sb = mips24_0[indsb]

In [7]:
z_lo = np.array([0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5])
z_hi = np.array([0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0])
z_mid = (z_lo+z_hi)/2.0
nz = len(z_mid)
m_lo = np.array([8.5,9.0, 9.5,10.0,10.5,11.0])
m_hi = np.array([9.0,9.5,10.0,10.5,11.0,13.0])
m_lo = np.array([8.5, 9.5,10.0,10.5,11.0])
m_hi = np.array([9.5,10.0,10.5,11.0,13.0])
m_mid = (m_lo+m_hi)/2.0
nm = len(m_mid)

In [8]:
path_5_pop = '/data/maps_cats_models/catalogs/multi_pop_lists/5pops/'
linear_mass = 1
if linear_mass == 1:
    linear_mass_suf = '_linear-'
else:
    linear_mass_suf = '_'
for iz in range(nz):
    for jm in range(nm):
        m_suf = ""+"-".join([str(m_lo[jm]),str(m_hi[jm])])
        z_suf = ""+"-".join([str(z_lo[iz]),str(z_hi[iz])])
        
        #Go through each population and see if there are objects in this m-z bin.  
        #If yes, write to file.  If no, write empty file?
        ##1: SF
        file_list ='list_5pop_z-'+z_suf+linear_mass_suf+'m-'+m_suf+'_sf.csv' 
        if linear_mass == 1:
            ind_mz_sf = np.where((z_peak_sf >= z_lo[iz]) & (z_peak_sf < z_hi[iz]) & (10**lmass_sf >= 10**m_lo[jm]) & (10**lmass_sf < 10**m_hi[jm]) )
        else:
            ind_mz_sf = np.where((z_peak_sf >= z_lo[iz]) & (z_peak_sf < z_hi[iz]) & (lmass_sf >= m_lo[jm]) & (lmass_sf < m_hi[jm]) )
        ntgal = np.size(ind_mz_sf)
        f = open(path_5_pop+file_list, 'w')
        if ntgal > 0:
            for kp1 in np.nditer(ind_mz_sf): 
                f.write(",".join([str(ra_sf[kp1]),str(dec_sf[kp1])]) + '\n')
        else:
            print 'no sources in '+file_list
        f.close()
        
        ##2: QT
        file_list ='list_5pop_z-'+z_suf+linear_mass_suf+'m-'+m_suf+'_qt.csv' 
        if linear_mass == 1:
            ind_mz_qt = np.where((z_peak_qt >= z_lo[iz]) & (z_peak_qt < z_hi[iz]) & (10**lmass_qt >= 10**m_lo[jm]) & (10**lmass_qt < 10**m_hi[jm]) )
        else:
            ind_mz_qt = np.where((z_peak_qt >= z_lo[iz]) & (z_peak_qt < z_hi[iz]) & (lmass_qt >= m_lo[jm]) & (lmass_qt < m_hi[jm]) )
        ntgal = np.size(ind_mz_qt)
        f = open(path_5_pop+file_list, 'w')
        if ntgal > 0:
            for kp2 in np.nditer(ind_mz_qt):
                f.write(",".join([str(ra_qt[kp2]),str(dec_qt[kp2])]) + '\n')
        else:
            print 'no sources in '+file_list
        f.close()
        
        ##3: AGN
        file_list ='list_5pop_z-'+z_suf+linear_mass_suf+'m-'+m_suf+'_agn.csv' 
        if linear_mass == 1:
            ind_mz_agn = np.where((z_peak_agn >= z_lo[iz]) & (z_peak_agn < z_hi[iz]) & (10**lmass_agn >= 10**m_lo[jm]) & (10**lmass_agn < 10**m_hi[jm]) )
        else:
            ind_mz_agn = np.where((z_peak_agn >= z_lo[iz]) & (z_peak_agn < z_hi[iz]) & (lmass_agn >= m_lo[jm]) & (lmass_agn < m_hi[jm]) )
        ntgal = np.size(ind_mz_agn)
        f = open(path_5_pop+file_list, 'w')
        if ntgal > 0:
            for kp3 in np.nditer(ind_mz_agn):
                f.write(",".join([str(ra_agn[kp3]),str(dec_agn[kp3])]) + '\n')
        else:
            print 'no sources in '+file_list
        f.close()
        
        ##4: SB
        file_list ='list_5pop_z-'+z_suf+linear_mass_suf+'m-'+m_suf+'_sb.csv' 
        if linear_mass == 1:
            ind_mz_sb = np.where((z_peak_sb >= z_lo[iz]) & (z_peak_sb < z_hi[iz]) & (10**lmass_sb >= 10**m_lo[jm]) & (10**lmass_sb < 10**m_hi[jm]) )
        else:
            ind_mz_sb = np.where((z_peak_sb >= z_lo[iz]) & (z_peak_sb < z_hi[iz]) & (lmass_sb >= m_lo[jm]) & (lmass_sb < m_hi[jm]) )
        ntgal = np.size(ind_mz_sb)
        f = open(path_5_pop+file_list, 'w')
        if ntgal > 0:
            for kp4 in np.nditer(ind_mz_sb):
                f.write(",".join([str(ra_sb[kp4]),str(dec_sb[kp4])]) + '\n')
        else:
            print 'no sources in '+file_list
        f.close()
        
        ##5: LOC
        file_list ='list_5pop_z-'+z_suf+linear_mass_suf+'m-'+m_suf+'_loc.csv' 
        if linear_mass == 1:
            ind_mz_loc = np.where((z_peak_loc >= z_lo[iz]) & (z_peak_loc < z_hi[iz]) & (10**lmass_loc >= 10**m_lo[jm]) & (10**lmass_loc < 10**m_hi[jm]) )
        else:
            ind_mz_loc = np.where((z_peak_loc >= z_lo[iz]) & (z_peak_loc < z_hi[iz]) & (lmass_loc >= m_lo[jm]) & (lmass_loc < m_hi[jm]) )
        ntgal = np.size(ind_mz_loc)
        f = open(path_5_pop+file_list, 'w')
        if ntgal > 0:
            for kp5 in np.nditer(ind_mz_loc):
                f.write(",".join([str(ra_loc[kp5]),str(dec_loc[kp5])]) + '\n')
        else:
            print 'no sources in '+file_list
        f.close()
    

no sources in list_5pop_z-0.0-0.5_linear-m-8.5-9.5_loc.csv
no sources in list_5pop_z-0.5-1.0_linear-m-8.5-9.5_loc.csv
no sources in list_5pop_z-0.5-1.0_linear-m-9.5-10.0_loc.csv
no sources in list_5pop_z-1.0-1.5_linear-m-8.5-9.5_sb.csv
no sources in list_5pop_z-1.0-1.5_linear-m-8.5-9.5_loc.csv
no sources in list_5pop_z-1.0-1.5_linear-m-9.5-10.0_loc.csv
no sources in list_5pop_z-1.0-1.5_linear-m-10.0-10.5_loc.csv
no sources in list_5pop_z-1.0-1.5_linear-m-10.5-11.0_loc.csv
no sources in list_5pop_z-1.0-1.5_linear-m-11.0-13.0_loc.csv
no sources in list_5pop_z-1.5-2.0_linear-m-8.5-9.5_sb.csv
no sources in list_5pop_z-1.5-2.0_linear-m-8.5-9.5_loc.csv
no sources in list_5pop_z-1.5-2.0_linear-m-9.5-10.0_sb.csv
no sources in list_5pop_z-1.5-2.0_linear-m-9.5-10.0_loc.csv
no sources in list_5pop_z-1.5-2.0_linear-m-10.0-10.5_loc.csv
no sources in list_5pop_z-1.5-2.0_linear-m-10.5-11.0_loc.csv
no sources in list_5pop_z-1.5-2.0_linear-m-11.0-13.0_loc.csv
no sources in list_5pop_z-2.0-2.5_linear-m-